# Conveyor Model Optimizer

Customers always enter in the first slot of the server 

In [1]:
using JuMP
using HiGHS
using Juniper

include("utils.jl") # demand_generator_mat, printTable, plotData
# include("CCParameters.jl") 

plotData (generic function with 1 method)

In [ ]:
# Parameters
horiz = 3           # total horizon
t = 1:horiz

# bounds
YM = 10              # max buffer legnth before dropping calls
XM = 6               # max queue length 
phiM = 4             # max adimission to queue
serM = 2             # number of servers
tserM = 3            # max service time


# iniital conditions
X0 = 3
Y0 = 2
L0 = 0
Z0 = 0

createDemands = false
if createDemands
    d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
    a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
else
    d_mat = demand_generator_mat(1, 20, 5, "uniform", 1)
    a_mat = demand_generator_mat(1,20,1,"uniform",0.5)
end

d = d_mat[1:horiz, 1];  # demand for incoming calls
a = a_mat[1:horiz, 1];  # abandonment for calls

In [ ]:
X = zeros(Int, horiz+1)     # current number of customers in queue x(k)
Y = zeros(Int, horiz+1)     # current number of customers in buffer y(k)  
Z = zeros(Int, horiz+1)     # custumers served  
L = zeros(Int, horiz+1)     # number of customers lost

n = zeros(Int, horiz+1)     # number of empty slots in the queue
q = zeros(Int, horiz+1)     # custumers entering queue
dr = zeros(Int, horiz+1)    # dropped due to full buffer

phi = zeros(Int, horiz+1)     # number of customers admitted to queue
Cin = zeros(Int, horiz+1)     # number of customers entering server
Cout = zeros(Int, horiz+1)    # number of customers leaving server

S = zeros(Int, horiz+1)                 # number of active servers
Sl = zeros(Int, horiz+1)                # number of free servers
Sst = zeros(Bool, horiz+1, serM)        # server status (0 - free, 1 - busy)
Sc = zeros(Bool, horiz+1, serM, tserM)  # server conveyor

Saux = zeros(Bool, horiz+1, serM)       # auxiliary server variable

default_input = zeros(Bool, serM, tserM)  # default input for servers
for i in 1:serM
    default_input[i, 1] = 1
end

transition_matrix = zeros(Bool, tserM, tserM)
for i in 1:tserM-1
    transition_matrix[i, i+1] = 1
end

In [ ]:
X[1] = X0
Y[1] = Y0
Z[1] = Z0
L[1] = L0

In [ ]:
for t in 1:horiz
    cc_lin_conv = Model(HiGHS.Optimizer)
    
    # For the integer constraint on variable b
    M1 = max(d[t], YM)+10; # must be larger than d[t] and n[t] 
    @variable(cc_lin_conv, b1 , Bin);

    M2 = max(X[t], serM)+10; # must be larger than x[k] and Sl[k]
    @variable(cc_lin_conv, b2 , Bin);

    @variable(cc_lin_conv, 0 <= XL[1:2] <= XM, Int)    
    @variable(cc_lin_conv, 0 <= YL[1:2] <= YM, Int)     
    @variable(cc_lin_conv, 0 <= ZL[1:2], Int)  
    @variable(cc_lin_conv, 0 <= LL[1:2], Int)         
                           
    @variable(cc_lin_conv, 0 <= nL <= YM, Int) 
    @variable(cc_lin_conv, 0 <= QL, Int)      
    @variable(cc_lin_conv, 0 <= drL, Int)   
    
    @variable(cc_lin_conv, 0 <= phiL <= phiM, Int)   
    @variable(cc_lin_conv, 0 <= CinL[1:2] <= serM, Int)   
    @variable(cc_lin_conv, 0 <= CoutL[1:2] <= serM, Int)    

    @variable(cc_lin_conv, 0 <= SL <= serM, Int)  
    @variable(cc_lin_conv, 0 <= SlL <= serM, Int)  
    @variable(cc_lin_conv, 0 <= SstL[1:serM], Bin)  
    @variable(cc_lin_conv, 0 <= ScL[2, 1:serM, 1:tserM], Bin) 
    @variable(cc_lin_conv, 0 <= SinL[1:serM, 1:tserM], Bin)
    @variable(cc_lin_conv, 0 <= SauxL[1:serM], Bin)

        
    # Initial conditions of buffer and queue for each optimization
    @constraint(cc_lin_conv, XL[1] == X[t])
    @constraint(cc_lin_conv, YL[1] == Y[t])
    @constraint(cc_lin_conv, ZL[1] == Z[t])
    @constraint(cc_lin_conv, CinL[1] == Cin[t])
    @constraint(cc_lin_conv, CoutL[1] == Cout[t])
    @constraint(cc_lin_conv, ScL[1, :, :] == Sc[t, :, :])  # server conveyor status

    # Problem constraints  
    #### DEVERIA TER UMA CONSTRAINT PARA PHI(K+1) <= Y(K)? ####
    @constraint(cc_lin_conv, XL[2] == XL[1] + phiL - a[t] - CinL)
    @constraint(cc_lin_conv, YL[2] == YL[1] + QL - phiL)
    @constraint(cc_lin_conv, ZL[2] == ZL[1] + CoutL)

    @constraint(cc_lin_conv, nL == YM - YL[1] + phiL)  # number of empty slots in the queue
    @constraint(cc_lin_conv, QL <= d[t])
    @constraint(cc_lin_conv, QL <= nL)
    @constraint(cc_lin_conv, QL >= d[t]-M1*b1 )
    @constraint(cc_lin_conv, QL >= nL-(1-b1)*M1)  

    @constraint(cc_nl_fobj_os, drL >= d[t]-nL)         
    @constraint(cc_nl_fobj_os, drL <= d[t]-QL) 

    @constraint(cc_lin_conv, SL == serM)              
    @constraint(cc_lin_conv, SstL == sums(ScL, dims=2))  
    @constraint(cc_lin_conv, SlL == SL - sum(SstL))    
    @constraint(cc_lin_conv, [i=1:serM, j=1:tser-1], SinL[i, j] == default_input[i, j] * SauxL[i])
    @constraint(cc_lin_conv, [i=1:serM], ScL[2, i, :] .== transition_matrix * ScL[1, i, :] + SinL[i, :])

    @constraint(cc_lin_conv, CinL[2] <= X[1])
    @constraint(cc_lin_conv, CinL[2] <= SlL)
    @constraint(cc_lin_conv, CinL[2] >= X[1]-M2*b2 )
    @constraint(cc_lin_conv, CinL[2] >= SlL-(1-b2)*M2)  
    @constraint(cc_lin_conv, CoutL[2] == sum(ScL[1,:,tserM]))  

    @constraint(cc_lin_conv, SauxL <= ones(Bool, serM)-SstL[1:SstL])  
    @constraint(cc_lin_conv, CinL[1] == sum(SauxL))  


    # Objective function
    @objective(cc_lin_conv, Max, CoutL[1] - drL - phiL); 
    
    JuMP.optimize!(cc_lin_conv)

    # Save computed values
    X[t+1] = JuMP.value(XL[2]);
    Y[t+1] = JuMP.value(YL[2]);
    Z[t+1] = JuMP.value(ZL[2]);


    n[t]   = JuMP.value(nL);
    Q[t]   = JuMP.value(QL);  
    dr[t]  = JuMP.value(drL);

    L[t+1] = JuMP.value(LL[2]);
    
    S[t] = JuMP.value(SL);


    phi[t] = JuMP.value(phiL);
    
    b1_opt[t] = JuMP.value(b1);
    b2_opt[t] = JuMP.value(b2);
    Cin[t+1] = JuMP.value(CinL[2]);
    Cout[t+1] = JuMP.value(CoutL[2]);
    Sc[t+1, :, :] = JuMP.value(ScL[2, :, :]);


    J[t] = objective_value(cc_nl_fobj_os) #Repeated line

    # Display results
    println("Optimal solution:")
    println("Objective value = ", objective_value(cc_lin_conv))

end

┌ Warning: Axis contains one element: 2. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[2]` instead of `2`.
└ @ JuMP.Containers ~/.julia/packages/JuMP/LKjRR/src/Containers/DenseAxisArray.jl:185


LoadError: KeyError: key 1 not found

In [46]:
ScL = zeros(serM, tserM)

2×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [47]:
sum(ScL, dims=2) # sum over the second dimension (tserM)

2×1 Matrix{Float64}:
 0.0
 0.0

In [45]:
serM

2